# Magazine Input Tutorial
We will be continuing our investigation of colors by exploring representation among magazine advertisments. Lets import some libraries that we will need for this activity.

This activity works best if run in Google Chrome.

In [ ]:
#@title <-- Import Libraries

# Used for loading our images and manipulating images.
import io
from base64 import b64decode, b64encode
import PIL.Image
from google.colab import files

# Used for image display and input. 
from IPython.display import HTML, Image, Javascript, clear_output
from google.colab.output import eval_js

# Used to hold data nd for plotting
import pandas as pd
import plotly.express as px
import plotly.io as pio

#@title <- Make sure to run this cell!

# Javascript (the programming language of browswer) code to create an image that
# you can select points on/
canvas_html = """
<canvas width=%d height=%d, id="canvas"></canvas>
<button id="finish">Finish</button>

<script>
var canvas = document.querySelector('#canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
var finish = document.querySelector('#finish')
var mouse = {x: 0, y: 0}
var selected_points = []

canvas.addEventListener('mousedown', e => {
  ctx.beginPath()
  let xCord = e.offsetX;
  let yCord = e.offsetY;
  ctx.beginPath(); //Start path
  selected_points.push({'x': xCord, 'y':yCord});
  ctx.arc(xCord -3 , yCord - 3, 6, 0, Math.PI * 2, true); // Draw a point using the arc function of the canvas with a point structure.
  ctx.fillStyle = "#FF0000";
  ctx.fill();
});
</script>
"""
# Javascript code to process event for when use clicks done.
finish_promise = \
"""
var data = new Promise(resolve=>{
  Promise.all(
    [
      finish.onclick = ()=> {
       resolve(canvas.toDataURL('image/png'))
      }
    ]
  )
})
"""
# Code to actually draw the canvas.
update_image = \
"""
var canvas = document.querySelector('#canvas');
console.log("Updating image");
console.log(canvas)
var newImage = new Image();
ctx.clearRect(0, 0, canvas.width, canvas.height);
ctx.beginPath();
newImage.src = "data:image/png;base64,{IMAGE_DATA}";
newImage.onload = function () {
    ctx.drawImage(newImage, 0, 0);
};

selected_points = [];
"""

# Upload Your Files
Run the cell below and select all the files that you want explore. TIP: Click on the top image in the folder, hold down shift, then click on the bottom image in the folder to select all.

In [ ]:
uploaded_files = files.upload()

# Point Selection
We will now select a point from every person in each one of the images provided. For each image, you should click on the RGB pixel that you want to sample. Once you have selected a sample from each person, click finish and the next image will be selected.

In [ ]:
# Create a pandas dataframe to hold all the selected points.
selected_points = pd.DataFrame()
# Display the image drawing canvas.
display(HTML(canvas_html % (600, 600, 5)))

# Display each uploaded file and allowpoint selection.
for i, raw_image in enumerate(uploaded_files.values()):
  # Load image and resize so that the largest axis is 600 pixels.
  image = PIL.Image.open(io.BytesIO(raw_image))
  width, height = image.size
  resize_ratio = max(max(width, height) / 600, 1)
  image = image.resize((int(width / resize_ratio), int(height / resize_ratio)))
  width, height = image.size

  # Convert image to a new format (called Base64) which we can display.
  buffer = io.BytesIO()
  image = image.convert("RGB")
  image.save(buffer, format="png") 
  encoded_image = b64encode(buffer.getvalue()).decode()

  # Get user input.
  display(Javascript(update_image.replace("{IMAGE_DATA}", encoded_image)))
  display(Javascript(finish_promise))
  eval_js(f"data")
  
  chosen_points = eval_js("selected_points")
  for point in chosen_points:
    x_selection = point['x']
    y_selection = point['y']
    if (0 <= x_selection and x_selection < width and
        0 <= y_selection and y_selection < height):
      r, g, b = image.getpixel((point['x'], point['y']))
      selected_points = selected_points.append(
          {"red": r, "green": g, "blue": b}, ignore_index=True)
clear_output()

# Your Points
Here are all the rgb point values that you sampled! Take a screen shot of the scatter plot and add it to the class slide show. Be sure to label your image.

In [ ]:
selected_points['color'] = selected_points.apply(
    lambda row: f"rgb({int(row['red'])},{int(row['green'])},{int(row['blue'])})",
    axis=1)
selected_points

,blue,green,red,color
0,156.0,178.0,222.0,"rgb(222,178,156)"
1,197.0,211.0,242.0,"rgb(242,211,197)"
2,124.0,140.0,192.0,"rgb(192,140,124)"
3,85.0,116.0,173.0,"rgb(173,116,85)"
4,113.0,129.0,169.0,"rgb(169,129,113)"
5,67.0,91.0,136.0,"rgb(136,91,67)"
6,160.0,174.0,219.0,"rgb(219,174,160)"
7,129.0,147.0,197.0,"rgb(197,147,129)"
8,114.0,161.0,209.0,"rgb(209,161,114)"
9,78.0,110.0,173.0,"rgb(173,110,78)"


In [ ]:
# Import plotly - a library for graphing in python
import plotly.graph_objects as go

def create_3d_scatter(selected_points):
  fig = go.Figure()

  # Create 3D scatter plot.
  fig.add_trace(dict(type='scatter3d',
                x=selected_points['red'],
                y=selected_points['green'],
                z=selected_points['blue'],
                mode='markers',
                marker=dict(color=list(selected_points['color']),
                            size=10)))

  # Set axis labels and dimensions.
  fig.update_layout(scene = dict(
                    xaxis_title='RED',
                    yaxis_title='GREEN',
                    zaxis_title='BLUE',
                    xaxis = dict(nticks=4, range=[0,255],),
                    yaxis = dict(nticks=4, range=[0,255],),
                    zaxis = dict(nticks=4, range=[0,255],))) 
  return fig    
fig = create_3d_scatter(selected_points)
pio.show(fig)

# Export and save our points
We will continue to explore these points in a later lesson! Export them below and save the text somewhere safe! Be sure to add a note clarifying if they are 'articles' or 'advertisements'.

In [ ]:
print(selected_points.to_json())

{"blue":{"0":156.0,"1":197.0,"2":124.0,"3":85.0,"4":113.0,"5":67.0,"6":160.0,"7":129.0,"8":114.0,"9":78.0,"10":139.0,"11":126.0,"12":120.0},"green":{"0":178.0,"1":211.0,"2":140.0,"3":116.0,"4":129.0,"5":91.0,"6":174.0,"7":147.0,"8":161.0,"9":110.0,"10":183.0,"11":152.0,"12":145.0},"red":{"0":222.0,"1":242.0,"2":192.0,"3":173.0,"4":169.0,"5":136.0,"6":219.0,"7":197.0,"8":209.0,"9":173.0,"10":228.0,"11":200.0,"12":192.0},"color":{"0":"rgb(222,178,156)","1":"rgb(242,211,197)","2":"rgb(192,140,124)","3":"rgb(173,116,85)","4":"rgb(169,129,113)","5":"rgb(136,91,67)","6":"rgb(219,174,160)","7":"rgb(197,147,129)","8":"rgb(209,161,114)","9":"rgb(173,110,78)","10":"rgb(228,183,139)","11":"rgb(200,152,126)","12":"rgb(192,145,120)"}}
